# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
citiesdf=pd.read_csv('../output_data/cities.csv')
citiesdf.head()

,City,Country,Date,Humidity,Cloudiness,Lat,Lng,Max Temp,Wind Speed
0,kaitangata,NZ,1.579821e+09,83.0,99.0,-46.28,169.85,59.70,12.95
1,guerrero negro,MX,1.579821e+09,48.0,0.0,27.98,-114.06,69.21,11.30
2,sharkan,RU,1.579821e+09,91.0,65.0,57.30,53.87,19.99,1.01
3,hithadhoo,MV,1.579821e+09,78.0,14.0,-0.60,73.08,83.66,12.59
4,kieta,PG,1.579821e+09,70.0,43.0,-6.22,155.63,84.11,4.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = citiesdf[["Lat", "Lng"]] 
figure_layout = {
    'width': '600px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout, map_type='TERRAIN')
heat_layer = gmaps.heatmap_layer(locations, weights=citiesdf['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [9]:
vacay_df=citiesdf[(citiesdf['Max Temp'] <= 85) & (citiesdf['Max Temp'] >= 70) & (citiesdf['Cloudiness'] < 30) &
                 (citiesdf['Humidity'] < 50) & (citiesdf['Wind Speed'] < 10)]
vacay_df.dropna(how='any')
vacay_df

,City,Country,Date,Humidity,Cloudiness,Lat,Lng,Max Temp,Wind Speed
13,general escobedo,MX,1.579821e+09,33.0,5.0,25.82,-100.33,78.80,5.82
98,keetmanshoop,NaN,1.579821e+09,31.0,0.0,-26.58,18.13,75.20,9.17
120,matamoros,MX,1.579821e+09,25.0,5.0,25.53,-103.25,73.40,5.21
522,yaan,NG,1.579821e+09,14.0,19.0,7.38,8.57,75.58,6.98
572,carnot,CF,1.579821e+09,16.0,28.0,4.94,15.87,71.11,8.37
585,santiago del estero,AR,1.579821e+09,38.0,0.0,-27.80,-64.26,81.00,9.98


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
vacay_df['Hotel Name']=''

radius = 5000
type = "hotel"
params = {"radius": radius, "type": type, "key": g_key}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

C:\Users\ktauf\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
for index, row in vacay_df.iterrows():
    params['location'] = (f"{row['Lat']},{row['Lng']}")
    response = requests.get(base_url, params=params).json()    
    try:
        vacay_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        next
vacay_df.iloc[1,1]='NA'
vacay_df

C:\Users\ktauf\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,City,Country,Date,Humidity,Cloudiness,Lat,Lng,Max Temp,Wind Speed,Hotel Name
13,general escobedo,MX,1.579821e+09,33.0,5.0,25.82,-100.33,78.80,5.82,Monterrey
98,keetmanshoop,NA,1.579821e+09,31.0,0.0,-26.58,18.13,75.20,9.17,Keetmanshoop
120,matamoros,MX,1.579821e+09,25.0,5.0,25.53,-103.25,73.40,5.21,Matamoros
522,yaan,NG,1.579821e+09,14.0,19.0,7.38,8.57,75.58,6.98,Iorjan
572,carnot,CF,1.579821e+09,16.0,28.0,4.94,15.87,71.11,8.37,Carnot
585,santiago del estero,AR,1.579821e+09,38.0,0.0,-27.80,-64.26,81.00,9.98,Santiago del Estero


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacay_df.iterrows()]
locations = vacay_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '600px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)
marker_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', 
                                  scale=3, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…